# Qrisp

In [ ]:
import qrisp

/Users/matija/.venv/fmf-qrisp/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Sreda: Uvod v Qrisp

<!-- - (5 min) zgodovina prehoda klasičnega računalništva od pretikanja kablov do danes, vzporednice s trenutnim stanjem kvantnih računalnikov
- (10 min) odprejo online Jupyter z nameščenim Qrispom in poženejo testni primer
- osnovni koraki v Qrispu, zapis vezij prek spremenljivk in funkcij, meritve, …
- Qrisp preizkusijo na vezjih, ki so jih do sedaj videli na šoli ter to primerjajo s pisanjem/risanjem v Qiskitu
- prikaz dodatnih kvantnih tipov (`QuantumBool`, `QuantumFloat`, …), operacij na njih, ter izračunanih vezij
- v Qrispu pišejo svoje račune in kratke programe ter gledajo, kakšna vezja dobijo
- pisanje kvantnih funkcij in samodejno dodajanje ancile, razračunanja, …
- pišejo svoje funkcije, simulirajo rezultate, in opazujejo ustvarjena vezja
- skupaj napišemo svoj lastni kvantni tip, npr. vržemo dve kvantni kocki in dobimo superpozicijo možnih vsot, pri obratu faze pa se nekateri izničijo -->

### Zgodovina prehoda klasičnega računalništva od pretikanja kablov do danes, vzporednice s trenutnim stanjem kvantnih računalnikov

V novicah se pogosto pojavljajo novice o rekordnih številih kubitov tehnoloških gigantov tipa [Google](https://blog.google/technology/research/google-willow-quantum-chip/), [IBM](https://www.ibm.com/roadmaps/quantum/), [Microsoft](https://news.microsoft.com/azure-quantum/), [Amazon](https://www.amazon.science/blog/amazon-announces-ocelot-quantum-chip), etc. Bistveno manj govora (če sploh) je namenjenega t.i. "software engineering tools", ki so prav tako ključnega pomena, kot klasični programerski jeziki, za eksekucijo 

Kako prevesti algoritme v jezik, ki ga kvantni računalnik sprejme, da izvrši račun ter, kako interpretiramo rezultat meritev? Kako lahko programiramo kvantni računalnik?

Do nedavnega je bil splošni konsenz, da se to počne z direktno manipulacijo kubitov prek dodajanja različnih vrat v kvantno vezje. Pri preprostih algoritmih, ki se jih lahko poganja z današnjimi [NISQ](https://en.wikipedia.org/wiki/Noisy_intermediate-scale_quantum_era) kvantni računalniki kvantna vezja ne predstavljajo prevelikega problema. Vendar kaj se zgodi, ko uporabnik želi več kubitov? Podobno vprašanje kot za širino vezij lahko postavimo tudi za globino vezij. Kako težko je voditi evidenco dodatnih t.i. "ancillary" qubits potrebnih za določene algoritemske primitive? 

Izkaže se, da precej. Dober dokaz je [kvantni backtracking algoritem](https://arxiv.org/pdf/2402.10060).

![Prepleteni kubiti](slikovna_gradiva/wire_chaos.png)

Sestavljanje vezij je neizbežno iz vidika učenja prvih principov kvantnega računalništva. Orodja kot so [Quirk](https://algassert.com/quirk#circuit={%22cols%22:[]}) nudijo možnost vizualizacije različnih aspektov (faza, usmeritev na Blochovi sferi, etc.), vendar počepnejo, ko se poskuša implementirati, kot tudi analizirati zahtevnejše, kompleksnejše, in "resource heavy" algoritme z vezji bistveno večjih globin in širin.

Podoben preskok kot preskok iz assemblerja v uporabniku prijaznejše programske jezike (povprašajte starejše po predavalnici), ponuja Qrisp, s katerim se ne osredotočamo zgolj na direktno manipulacijo kubitov v sklopu kvantnih vezij, temveč skrijemo kubite in manipulacijo le-teh za višjenivojske abstrakcije, ki omogočijo programiranje v sklopu spremenljivk in funkcij.

![Drake](slikovna_gradiva/qrisp_drake.png)

![Qrisp logo](slikovna_gradiva/logo_extended.png)

Qrisp je odprtokodni visokonivojski kvantni programski jezik za programiranje kvantnih računalnikov. Zaradi drugačnega pristopa h kreiranju vezij in direktnih vzporednic konceptov v klasičnem programiranju - Qrisp uvaja abstrakcije, ki omogočajo pisanje kvantnih programov z uporabo konceptov, znanih iz klasičnega programiranja, kot so spremenljivke, zanke, pogojniki in funkcije. Zaradi tega so izhodna vezja učinkovitejša ter koda modularna, kar omogoča sodelovanje večih programerjev brez potrebnega usklajevanja.

![prepisovanje dveh kompozicij](slikovna_gradiva/analogija_z_glasbo.png)

### Kvantne spremenljivke: [QuantumVariable](https://www.qrisp.eu/reference/Core/QuantumVariable.html)

QuantumVariable je kvantni ekvivalent klasični spremenljivki. Predstavlja register skupaj združenih kubitov, kar olajša življenje (morda ne na prvi pogled prek preprostih primerov) pri implementaciji algoritmov - skriva zapletenost upravljanja kubitov.

Je osnoven kvantni tip višje ravni v Qrispu - preostali kvantni tipi, nekatere srečamo kaj kmalu (QuantumFloat, QuantumBool), dedujejo iz QuantumVariable.

Kar začnimo z deklaracijo [kvantne spremenljivke](https://www.qrisp.eu/reference/Core/QuantumVariable.html) s štirimi kubiti.

In [ ]:
from qrisp import QuantumVariable

alica = QuantumVariable(4, name="alica")

Z `name="alica"` smo deklarirano kvantno spremenljivko tudi poimenovali zavoljo lažje predstave tega dotičnega registra kubitov. 

In [ ]:
print(alica)

In [ ]:
print(alica.size)

Kvantno vezje lahko vselej izrišemo s klicom metode `.qs` - [QuantumSession](https://www.qrisp.eu/reference/Core/QuantumSession.html), v katerem se vodi evidenco do sedaj izvedenih operacij z ozirom na kvantno vezje.

In [ ]:
print(alica.qs)

Znotraj kvantne spremenljivke lahko dostopamo tudi do posamičnih kubitov preko operacije oglatega oklepaja.

In [ ]:
prvi_kubit = alica[0]
zadnja_dva = alica[:2]

#### Kvantna vrata

Tekom šole ste se že seznanili  z različnimi kvantnimi vrati, kot so recimo x - not, h - Hadamard ter cx - kontroliran not (CNOT).

Ta vrata, kot tudi [obilica drugih](https://www.qrisp.eu/reference/Circuit%20Manipulation/QuantumCircuit.html#operation-application-methods) lahko enostavno apliciramo na kvantno spremenljivko.

In [ ]:
from qrisp import x, h, cx

x(prvi_kubit)
h(alica[1])
cx(zadnja_dva)

Izvedene operacije lahko, tako kot prej, vizualiziramo s printom `.qs`.

In [ ]:
print(alica.qs)

Izpišemo lahko tudi t.i. [statevector](https://www.qrisp.eu/reference/Core/generated/qrisp.QuantumSession.statevector.html#qrisp.QuantumSession.statevector) (vektor stanja?).

In [ ]:
print(alica.qs.statevector)

Preizkusimo naučene koncepte na malo bolj "uporabnem" primeru priprave t.i. [GHZ stanja](https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state), ki priravi stanje bodisi samih ničel, bodisi samih enic.

![analogija s kovanci](slikovna_gradiva/GHZ_kovanci.png)

V primeru kvantne spremenljivke s tremi kubiti bi takšno GHZ stanje izgledalo nekako takole:$$GHZ_3=\frac{1}{\sqrt2}(\ket{000}+\ket{111}),$$

kjer indeks 3 sugerira število kubitov. 

Imate kakšno idejo kako bi se tega lotili zgolj z že omenjenimi kvantnimi vrati?

![malo kasneje](slikovna_gradiva/malo_kasneje.png)

qv = QuantumVariable(3)

def GHZ(qv):
    h(qv[0])
    for i in range (1, qv.size)
        cx(qv[0], qv[i])

Komentar...

In [ ]:
GHZ = GHZ(qv)
print(qv)

In [ ]:
print(qv.qs)

In [ ]:
print(qv.qs.statevector())
print(qv.qs.depth())
print(qv.qs.cnot_count())

### Kvantni tipi: [QuantumFloat](https://www.qrisp.eu/reference/Quantum%20Types/QuantumFloat.html)
binarni zapis števil - callback

#### Računanje s kvantnimi floati - [prefix aritmetika](https://www.qrisp.eu/reference/Primitives/Prefix%20arithmetic.html)

##### superpozicija 101

#### racunanje 2+superpozicija(3, 5)

### Kvantni tipi: [QuantumBool](https://www.qrisp.eu/reference/Quantum%20Types/QuantumBool.html#quantumbool) 

#### Logični & in logični ^

#### Primerjava s kvantnimi floati

### Avtomatsko kvantno ["odračunanje"](https://www.qrisp.eu/reference/Core/Uncomputation.html)

### Kvantne funkcije

#### QPE (v četrtek?)

#### Shor (v četrtek)
Daste mi številko in jaz bom to čarobno faktoriziral

## Četrtek: Deutsch-Jozsev algoritem

<!-- - motivacija Deutsch-Jozsevega problema prek igrice, kjer poskušajo s čim manj vprašanji ugotoviti, kakšne vrste število sem si izmislil
- dopolnijo predlogo klasično rešitve v Pythonu ter v njej preizkušajo različne funkcije (konstantno, levi-desni biti, sodi-lihi biti, …)
- prepis klasične rešitve v Qrisp in preizkus z različnimi funkcijami (tu in po vsakem od naslednjih korakov do algoritma)
- skupaj pridemo do tega, da bi računali s superpozicijami, zato vhodne kubite pripravimo s Hadamardom
- spomnimo se, da je s ponovno uporabo Hadamarda treba povrniti vhodne kubite
- skupaj pridemo do tega, da želimo vrednosti izničiti, zato izhodni kubit pripravimo s Hadamardom
- kaj so naslednji koraki v programiranju (Bernstein-Vazirani, QPE, Grover, Shor, kvantne simulacije, …) -->

In [ ]:
from qrisp import QuantumBool, QuantumFloat, h, cx, x, auto_uncompute

N = 2

In [ ]:
def sodo(vhod, izhod):
    cx(vhod[0], izhod)

def konstantno0(vhod, izhod):
    pass

def konstantno1(vhod, izhod):
    x(izhod)

@auto_uncompute
def v_prvi_polovici(vhod, izhod):
    rezultat = vhod < 2 ** (N - 1)
    cx(rezultat, izhod)

In [ ]:
vhod = QuantumFloat(N)
izhod = QuantumBool(qs=vhod.qs)
x(izhod)

h(vhod)

h(izhod)

v_prvi_polovici(vhod, izhod)

h(vhod)

vhod.qs.statevector()

Simulating 5 qubits.. |                                                      | [  0%]

(|0>*|True> + |1>*|True> + |2>*|False> + |3>*|False>)/2

## Kam naprej?